# Anomalo vs Normal

In [23]:
import pandas as pd
import numpy as np
import os
import shutil
import cv2

import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from matplotlib import cm
import matplotlib.patches as patches

from PIL import Image, ImageOps

print("Listo !!")

In [7]:
info=pd.read_csv('../input/mias-mammography/Info.txt',sep=" ")
info.drop(columns=['Unnamed: 7'],inplace=True)
info=info.loc[~info.REFNUM.duplicated(),:]
info.loc[info.CLASS=="NORM","Tipo"]="Normal"
info.loc[info.SEVERITY=="B","Tipo"]="Anomalo"
info.loc[info.SEVERITY=="M","Tipo"]="Anomalo"
eliminar=info.loc[ pd.isna(info.RADIUS) & (info.Tipo!="Normal"),:].REFNUM.to_list()
info=info.loc[~info.REFNUM.isin(eliminar),:]

info.index=info.REFNUM
info.Tipo.value_counts()

In [9]:
frec=info.Tipo.value_counts()

fig=plt.figure(figsize=(7,5))
plt.bar(frec.index,frec.values)
plt.title('Distribución de imágenes',fontsize=15)
plt.ylabel('Número de imágenes',fontsize=15)
plt.xlabel('Categorías',fontsize=15)

for absoluto,categoria in zip(frec.values,range(2)):
    pct=absoluto/frec.sum()*100
    pct=round(pct,2)
    plt.text(categoria-0.1,absoluto-8,str(pct)+'%',fontsize=12)
    
plt.savefig('Distribucion.png')
plt.show()

In [10]:
info_balanceado=info.drop(info.loc[info.Tipo=="Normal",:].sample(n=95,random_state=211).index)
info_balanceado.Tipo.value_counts()

In [11]:
frec=info_balanceado.Tipo.value_counts()

fig=plt.figure(figsize=(7,5))
plt.bar(frec.index,frec.values)
plt.title('Distribución de imágenes',fontsize=15)
plt.ylabel('Número de imágenes',fontsize=15)
plt.xlabel('Categorías',fontsize=15)

for absoluto,categoria in zip(frec.values,range(2)):
    pct=absoluto/frec.sum()*100
    pct=round(pct,2)
    plt.text(categoria-0.1,absoluto-8,str(pct)+'%',fontsize=12)
    
plt.savefig('Distribucion2.png')
plt.show()

In [14]:
i=1
plt.figure(figsize=(8, 5))
plt.suptitle('Imágenes de muestra',y=0.92,fontsize=15)

for tipo in ["Normal","Anomalo"]:
    
    muestras=info.Tipo[info.Tipo==tipo].sample(n=1,random_state=11).index    
    
    for muestra in muestras:
        
        if tipo=="Anomalo":
            
            img_path="../input/mias-mammography/all-mias/"+muestra+".pgm"
            
            cord_x=int(info.X[info.REFNUM==muestra])
            cord_y=1024-int(info.Y[info.REFNUM==muestra])
            radius=int(info.RADIUS[info.REFNUM==muestra])
                        
            img = cv2.imread(img_path)
            
            plt.subplot(1,2,i)
            plt.imshow(img)
            
            ax = plt.gca()
            rect = patches.Rectangle((cord_x-radius,cord_y-radius), radius*2, radius*2, linewidth=1, edgecolor='r', facecolor='none')
            ax.add_patch(rect)
            
            plt.title(tipo)
            i=i+1
            
        else:
            
            img_path="../input/mias-mammography/all-mias/"+muestra+".pgm"
            
            img = cv2.imread(img_path)
            plt.subplot(1,2,i)
            plt.imshow(img)
            plt.title(tipo)
            i=i+1

plt.savefig('muestras.png')            
plt.show()

In [15]:
# Aumentar datos
def aumentar_datos(input_img,path_output):
    Imagen = Image.open(input_img)
    volteo_v = ImageOps.flip(Imagen)
    volteo_v.save(path_output+"_vflip"+".png")
    volteo_h = ImageOps.mirror(Imagen)
    volteo_h.save(path_output+"_hflip"+".png")
    
    for angulo in range(30,345,30):
        rotacion = Imagen.rotate(angulo)
        rotacion.save(path_output+"_"+str(angulo)+".png")
        volteo_v = ImageOps.flip(rotacion)
        volteo_v.save(path_output+"_"+str(angulo)+"_vflip"+".png")
        volteo_h = ImageOps.mirror(rotacion)
        volteo_h.save(path_output+"_"+str(angulo)+"_hflip"+".png")
        

def procesado(ids, clip=5, interpolacion=cv2.INTER_AREA, biopsia=True):
    
    path_img="../input/mias-mammography/all-mias/"+ids+".pgm"
    
    img=cv2.imread(path_img,cv2.IMREAD_GRAYSCALE)
    
    cord_x, cord_y, radius, tipo = info.loc[ids,info.columns[4:8]]
    
    if tipo!="Normal":
        
        if biopsia==True:
            cord_x=int(cord_x)
            cord_y=1024-int(cord_y)
            radius=int((150/radius)*radius)
            
            inf_cord_y=cord_y-radius
            sup_cord_y=cord_y+radius
            inf_cord_x=cord_x-radius
            sup_cord_x=cord_x+radius
            
            if sup_cord_y>1024:
                inf_cord_y=inf_cord_y-(sup_cord_y-1024)
                sup_cord_y=1024
                
            if inf_cord_y<0:
                sup_cord_y=sup_cord_y+abs(inf_cord_y)
                inf_cord_y=0
                
            if sup_cord_x>1024:
                inf_cord_x=inf_cord_x-(sup_cord_x-1024)
                sup_cord_x=1024
                
            if inf_cord_x<0:
                sup_cord_x=sup_cord_x+abs(inf_cord_x)
                inf_cord_x=0
                
            img2=img[inf_cord_y:sup_cord_y, inf_cord_x:sup_cord_x]
        else:
            blur = cv2.GaussianBlur(img,(5,5),0)
            _, breast_mask = cv2.threshold(blur,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
            cnts, _ = cv2.findContours(breast_mask.astype(np.uint8), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
            cnt = max(cnts, key = cv2.contourArea)
            x, y, w, h = cv2.boundingRect(cnt)
            img2=img[y:y+h, x:x+w]       
    else:
        
        blur = cv2.GaussianBlur(img,(5,5),0)
        _, breast_mask = cv2.threshold(blur,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
        cnts, _ = cv2.findContours(breast_mask.astype(np.uint8), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        cnt = max(cnts, key = cv2.contourArea)
        x, y, w, h = cv2.boundingRect(cnt)
               
        if biopsia==True:
            crop_img=img[y:y+h, x:x+w]
            centro=(int(crop_img.shape[0]/2),int(crop_img.shape[1]/2))
            img2=crop_img[(centro[0]-120):(centro[0]+120),(centro[1]-120):(centro[1]+120)]            
        else:
            img2=img[y:y+h, x:x+w]
            
    clahe = cv2.createCLAHE(clipLimit = clip)
    cl = clahe.apply(np.array(img2, dtype=np.uint8))
    cl=cv2.resize(cl,(224,224),interpolation = interpolacion)
    img2=np.dstack([cl,cl,cl])
    
    return(img2)

print("Listo!!")

In [16]:
def zoom(ids):
    
    
    path_img="../input/mias-mammography/all-mias/"+ids+".pgm"
    
    img=cv2.imread(path_img,cv2.IMREAD_GRAYSCALE)
    
    cord_x, cord_y, radius, tipo = info.loc[ids,info.columns[4:8]]
    
    biopsia=True
    interpolacion=cv2.INTER_AREA
    
    
    if tipo!="Normal":
        
        if biopsia==True:
            cord_x=int(cord_x)
            cord_y=1024-int(cord_y)
            radius=int((150/radius)*radius)
            
            inf_cord_y=cord_y-radius
            sup_cord_y=cord_y+radius
            inf_cord_x=cord_x-radius
            sup_cord_x=cord_x+radius
            
            if sup_cord_y>1024:
                inf_cord_y=inf_cord_y-(sup_cord_y-1024)
                sup_cord_y=1024
                
            if inf_cord_y<0:
                sup_cord_y=sup_cord_y+abs(inf_cord_y)
                inf_cord_y=0
                
            if sup_cord_x>1024:
                inf_cord_x=inf_cord_x-(sup_cord_x-1024)
                sup_cord_x=1024
                
            if inf_cord_x<0:
                sup_cord_x=sup_cord_x+abs(inf_cord_x)
                inf_cord_x=0
                
            img2=img[inf_cord_y:sup_cord_y, inf_cord_x:sup_cord_x]

    else:
        
        blur = cv2.GaussianBlur(img,(5,5),0)
        _, breast_mask = cv2.threshold(blur,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
        cnts, _ = cv2.findContours(breast_mask.astype(np.uint8), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        cnt = max(cnts, key = cv2.contourArea)
        x, y, w, h = cv2.boundingRect(cnt)
               
        if biopsia==True:
            crop_img=img[y:y+h, x:x+w]
            centro=(int(crop_img.shape[0]/2),int(crop_img.shape[1]/2))
            img2=crop_img[(centro[0]-120):(centro[0]+120),(centro[1]-120):(centro[1]+120)]            
            
    img2=cv2.resize(img2,(224,224),interpolation = interpolacion)
    img2=np.dstack([img2,img2,img2])
        
    return(img2)

print("Listo!!")

In [17]:
ids="mdb005"

imagen_original=cv2.imread("../input/mias-mammography/all-mias/"+ids+".pgm")
imagen_zoom=zoom(ids)
imagen_procesada=procesado(ids,biopsia=True,clip=2)

fig=plt.figure(figsize=(15,5))

plt.subplot(1,3,1)
plt.imshow(imagen_original);
plt.title("Original")

plt.subplot(1,3,2)
plt.imshow(imagen_zoom)
plt.title("Zoom")

plt.subplot(1,3,3)
plt.imshow(imagen_procesada)
plt.title("CLAHE")

plt.savefig('procesado.png')            
plt.show();

In [18]:
#Particiones

from sklearn.model_selection import train_test_split

train, test = train_test_split(info_balanceado, test_size=0.5, random_state=42,stratify=info_balanceado.Tipo)

print("Listo!!!")

In [19]:
print(train.Tipo.value_counts(),"\n")
print(test.Tipo.value_counts(),"\n")

In [20]:
frec_A=train.Tipo.value_counts()
frec_B=test.Tipo.value_counts()

fig=plt.figure(figsize=(12,5))

plt.subplot(1,2,1)

plt.bar(frec_A.index,frec_B.values)
plt.title('Set Train: Distribución de imágenes',fontsize=15)
plt.ylabel('Número de imágenes',fontsize=15)
plt.xlabel('Categorías',fontsize=15)

for absoluto,categoria in zip(frec_A.values,range(2)):
    pct=absoluto/frec_A.sum()*100
    pct=round(pct,2)
    plt.text(categoria-0.15,absoluto-8,str(pct)+'%',fontsize=12)
    
plt.subplot(1,2,2)

plt.bar(frec_B.index,frec_B.values)
plt.title('Set Test: Distribución de imágenes',fontsize=15)
#plt.ylabel('Número de imágenes',fontsize=15)
plt.xlabel('Categorías',fontsize=15)

for absoluto,categoria in zip(frec_B.values,range(2)):
    pct=absoluto/frec_B.sum()*100
    pct=round(pct,2)
    plt.text(categoria-0.15,absoluto-8,str(pct)+'%',fontsize=12)
    
plt.savefig('Particiones.png')
plt.show()

In [21]:
### Aumentar datos
def aumentar_datos(input_img,path_output):
    Imagen = Image.open(input_img)
    volteo_v = ImageOps.flip(Imagen)
    volteo_v.save(path_output+"_vflip"+".png")
    volteo_h = ImageOps.mirror(Imagen)
    volteo_h.save(path_output+"_hflip"+".png")
    
    for angulo in range(30,345,30):
        rotacion = Imagen.rotate(angulo)
        rotacion.save(path_output+"_"+str(angulo)+".png")
        volteo_v = ImageOps.flip(rotacion)
        volteo_v.save(path_output+"_"+str(angulo)+"_vflip"+".png")
        volteo_h = ImageOps.mirror(rotacion)
        volteo_h.save(path_output+"_"+str(angulo)+"_hflip"+".png")
        

def procesado(ids, clip=5, interpolacion=cv2.INTER_AREA, biopsia=True):
    
    path_img="../input/mias-mammography/all-mias/"+ids+".pgm"
    
    img=cv2.imread(path_img,cv2.IMREAD_GRAYSCALE)
    
    cord_x, cord_y, radius, tipo = info.loc[ids,info.columns[4:8]]
    
    if tipo!="Normal":
        
        if biopsia==True:
            cord_x=int(cord_x)
            cord_y=1024-int(cord_y)
            radius=int((150/radius)*radius)
            
            inf_cord_y=cord_y-radius
            sup_cord_y=cord_y+radius
            inf_cord_x=cord_x-radius
            sup_cord_x=cord_x+radius
            
            if sup_cord_y>1024:
                inf_cord_y=inf_cord_y-(sup_cord_y-1024)
                sup_cord_y=1024
                
            if inf_cord_y<0:
                sup_cord_y=sup_cord_y+abs(inf_cord_y)
                inf_cord_y=0
                
            if sup_cord_x>1024:
                inf_cord_x=inf_cord_x-(sup_cord_x-1024)
                sup_cord_x=1024
                
            if inf_cord_x<0:
                sup_cord_x=sup_cord_x+abs(inf_cord_x)
                inf_cord_x=0
                
            img2=img[inf_cord_y:sup_cord_y, inf_cord_x:sup_cord_x]
        else:
            blur = cv2.GaussianBlur(img,(5,5),0)
            _, breast_mask = cv2.threshold(blur,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
            cnts, _ = cv2.findContours(breast_mask.astype(np.uint8), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
            cnt = max(cnts, key = cv2.contourArea)
            x, y, w, h = cv2.boundingRect(cnt)
            img2=img[y:y+h, x:x+w]       
    else:
        
        blur = cv2.GaussianBlur(img,(5,5),0)
        _, breast_mask = cv2.threshold(blur,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
        cnts, _ = cv2.findContours(breast_mask.astype(np.uint8), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        cnt = max(cnts, key = cv2.contourArea)
        x, y, w, h = cv2.boundingRect(cnt)
               
        if biopsia==True:
            crop_img=img[y:y+h, x:x+w]
            centro=(int(crop_img.shape[0]/2),int(crop_img.shape[1]/2))
            img2=crop_img[(centro[0]-120):(centro[0]+120),(centro[1]-120):(centro[1]+120)]            
        else:
            img2=img[y:y+h, x:x+w]
            
    clahe = cv2.createCLAHE(clipLimit = clip)
    cl = clahe.apply(np.array(img2, dtype=np.uint8))
    cl=cv2.resize(cl,(224,224),interpolation = interpolacion)
    img2=np.dstack([cl,cl,cl])
    
    return(img2)

print("Listo!!")

In [ ]:
os.makedirs("./Set/train/Normal/")
os.makedirs("./Set/train/Anomalo/")

os.makedirs("./Set/test/Normal/")
os.makedirs("./Set/test/Anomalo/")

for ids in train.index:
    img_procesada=procesado(ids,biopsia=True,clip=1)    
    Tipo=info.loc[ids,:].Tipo
    
    cv2.imwrite("./temp.png",img_procesada)
        
    if Tipo=="Normal":
        aumentar_datos("./temp.png","./Set/train/Normal/"+ids)
        
    else:
        aumentar_datos("./temp.png","./Set/train/Anomalo/"+ids)
        
    os.remove("./temp.png")   


for ids in test.index:
    img_procesada=procesado(ids, biopsia=True, clip=1)    
    Tipo=info.loc[ids,:].Tipo
    
    if Tipo=="Normal":
        cv2.imwrite("./Set/test/Normal/"+ids+".png",img_procesada)
        
    else:
        cv2.imwrite("./Set/test/Anomalo/"+ids+".png",img_procesada)
    
print("Listo!!!")

In [27]:
print(len(os.listdir(("./Set/train/Normal/"))))
print(len(os.listdir(("./Set/test/Normal/"))))

print(len(os.listdir(("./Set/train/Anomalo/"))))
print(len(os.listdir(("./Set/test/Anomalo/"))))

In [28]:
##Ejemplo de rotaciones
ids="mdb021"
imagenes=[ i for i in os.listdir("./Set/train/Anomalo/") if i.find(ids)>=0]
imagenes=imagenes[0:10]

i=1
plt.figure(figsize=(15, 8))
plt.suptitle('Data augmentation',y=0.92,fontsize=15)
for imagen in imagenes:
    plt.subplot(2,5,i)
    plt.imshow(plt.imread("./Set/train/Anomalo/"+imagen))
    i=i+1
plt.savefig('Augmentation.png')
plt.show() 

In [29]:
import tensorflow as tf

from tensorflow.keras.preprocessing.image import ImageDataGenerator  

from tensorflow.keras.layers import Flatten,Dense, Dropout, Activation, BatchNormalization

from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras import Sequential, Model

train_path='./Set/train/'
test_path='./Set/test/'

print("listo!!!")

In [30]:
resnet50 = tf.keras.applications.resnet50

train_datagen = ImageDataGenerator(
    preprocessing_function=resnet50.preprocess_input,
    validation_split=1/4,
    height_shift_range=0.1,
    width_shift_range=0.1,
    rotation_range=15)

train_generator=train_datagen.flow_from_directory(
    directory=train_path,
    classes=["Normal","Anomalo"],
    target_size = (224, 224), 
    shuffle=True,
    subset="training")

val_generator=train_datagen.flow_from_directory(    
    directory=train_path,
    classes=["Normal","Anomalo"],
    target_size = (224, 224), 
    shuffle=True,
    subset="validation")

In [ ]:
Resnet = resnet50.ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224,3), pooling="max")

from keras import backend as K

def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall


for layer in Resnet.layers:
    if layer.name.startswith('conv5'):
        layer.trainable=True
    else:
        layer.trainable=False


x = Resnet.output
x = Flatten()(x)
x = Dense(64,use_bias=False)(x)
x = Activation('relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.55)(x)

x = Dense(16,use_bias=False)(x)
x = Activation('relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.55)(x)
predictions = Dense(2, activation="softmax")(x)

model_resnet50 = Model(inputs=Resnet.inputs, outputs=predictions)
optimizador=tf.keras.optimizers.Adam(learning_rate=1e-4)
perdida=tf.keras.losses.CategoricalCrossentropy()



model_resnet50.compile(optimizer=optimizador,loss=perdida,metrics=['accuracy',tf.keras.metrics.AUC(),recall_m])

checkpoint_cb = ModelCheckpoint("model_resnet50.h5", monitor='val_accuracy',mode='max',save_best_only=True)
early_stopping_cb = EarlyStopping(patience=5,monitor='val_accuracy', mode='max',restore_best_weights=True)

history = model_resnet50.fit(train_generator,
                     epochs=100,
                     validation_data=val_generator,
                     callbacks=[checkpoint_cb,early_stopping_cb],verbose=1)

In [ ]:
metrics=history.history

plt.figure(figsize=(15, 5))

plt.subplot(1,2,1)
plt.plot(metrics['accuracy'])
plt.plot(metrics['val_accuracy'])
plt.title('accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='lower right')
    
plt.subplot(1,2,2)
plt.plot(metrics['loss'])
plt.plot(metrics['val_loss'])
plt.title('loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper right')

plt.show()

In [ ]:
val_generator=train_datagen.flow_from_directory(
    directory=train_path,
    classes=["Normal","Anomalo"],
    target_size = (224, 224), 
    shuffle=False,
    subset="validation")

from sklearn.metrics import accuracy_score, precision_score, recall_score ,f1_score, confusion_matrix, classification_report

prob_pred=model_resnet50.predict(val_generator)
clas_pred=prob_pred.argmax(axis=1)
clas_test=val_generator.classes

print(classification_report(clas_test,clas_pred))


ajuste=accuracy_score(clas_test,clas_pred).round(4)

matriz_confusion=confusion_matrix(y_true=clas_test,
                                  y_pred=clas_pred)

print("\nAjuste del test : {}".format(ajuste))
pd.DataFrame(matriz_confusion,
             columns=val_generator.class_indices.keys(),
             index=val_generator.class_indices.keys())

In [ ]:
test_datagen = ImageDataGenerator(
    preprocessing_function=resnet50.preprocess_input)

test_generator=test_datagen.flow_from_directory(
    directory=test_path,
    classes=["Normal","Anomalo"],
    target_size = (224, 224), 
    shuffle=False)

prob_pred=model_resnet50.predict(test_generator)
clas_pred=prob_pred.argmax(axis=1)
clas_test=test_generator.classes


ajuste=accuracy_score(clas_test,clas_pred).round(4)

matriz_confusion=confusion_matrix(y_true=clas_test,
                                  y_pred=clas_pred)

print("\nAjuste del test : {}".format(ajuste))

pd.DataFrame(matriz_confusion,
             columns=test_generator.class_indices.keys(),
             index=test_generator.class_indices.keys())

In [ ]:
# Salvar

metrics=history.history

import pickle

f = open("metrics_model_resnet50.pkl","wb")
pickle.dump(metrics,f)
f.close()

In [ ]:
# Cargar

path="../input/resultados/model_resnet50.h5"
modelo = tf.keras.models.load_model(path)

import pickle
path='../input/resultados/metrics_model_resnet50.pkl'
with open(path, 'rb') as f:
    metrics= pickle.load(f)

In [ ]:
test_datagen = ImageDataGenerator(
    preprocessing_function=resnet50.preprocess_input)

test_generator=test_datagen.flow_from_directory(
    directory=test_path,
    classes=["Normal","Anomalo"],
    target_size = (224, 224), 
    shuffle=False)

prob_pred=modelo.predict(test_generator)

predicciones=pd.DataFrame(prob_pred,columns=["Prob_Normal","Prob_Anomalo"])
predicciones["Clase_Observada"]=test_generator.classes
predicciones["Clase_Predicha"]=prob_pred.argmax(axis=1)
predicciones["rutas"]=test_generator.filepaths

In [ ]:
def get_img_array(img_path, size):
    img = tf.keras.preprocessing.image.load_img(img_path, target_size=size)
    array = tf.keras.preprocessing.image.img_to_array(img)
    array = np.expand_dims(array, axis=0)
    return array

def make_gradcam_heatmap(img_array, model, last_conv_layer_name, pred_index=None):
    # Primero, creamos un modelo que asigna la imagen de entrada las activaciones de la última capa de conv, 
    # así como a las predicciones de salida.

    grad_model = tf.keras.models.Model(
        [model.inputs], [model.get_layer(last_conv_layer_name).output, model.output]
    )

    # Luego, calculamos el gradiente de la clase predicha superior para nuestra imagen de entrada 
    # con respecto a las activaciones de la última capa de conv.

    with tf.GradientTape() as tape:
        last_conv_layer_output, preds = grad_model(img_array)
        if pred_index is None:
            pred_index = tf.argmax(preds[0])
        class_channel = preds[:, pred_index]

    # Este es el gradiente de la neurona de salida (superior predicha o elegida) 
    # con respecto al mapa de características de salida de la última capa de conv.
    grads = tape.gradient(class_channel, last_conv_layer_output)

    # Este es un vector donde cada entrada es la intensidad media del gradiente 
    # sobre un canal de mapa de características específico
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))

    # Multiplicamos cada canal en la matriz del mapa de características por "qué tan importante es este canal" 
    # con respecto a la clase predicha superior y luego sumamos todos los canales 
    # para obtener la activación de la clase del mapa de calor.
    last_conv_layer_output = last_conv_layer_output[0]
    heatmap = last_conv_layer_output @ pooled_grads[..., tf.newaxis]
    heatmap = tf.squeeze(heatmap)

    # Para fines de visualización, también normalizaremos el mapa de calor entre 0 y 1
    
    heatmap = tf.maximum(heatmap, 0) / tf.math.reduce_max(heatmap)
    return heatmap.numpy()

def display_gradcam(img_path, heatmap, alpha=0.4):


    img = tf.keras.preprocessing.image.load_img(img_path)
    img = tf.keras.preprocessing.image.img_to_array(img)

    heatmap = np.uint8(255 * heatmap)

    jet = cm.get_cmap("jet")

    jet_colors = jet(np.arange(256))[:, :3]
    jet_heatmap = jet_colors[heatmap]

    jet_heatmap = tf.keras.preprocessing.image.array_to_img(jet_heatmap)
    jet_heatmap = jet_heatmap.resize((img.shape[1], img.shape[0]))
    jet_heatmap = tf.keras.preprocessing.image.img_to_array(jet_heatmap)

    superimposed_img = jet_heatmap * alpha + img
    superimposed_img = tf.keras.preprocessing.image.array_to_img(superimposed_img)

    return(superimposed_img)

In [ ]:
seleccion=predicciones.loc[(predicciones.Clase_Observada==predicciones.Clase_Predicha) & (predicciones.Clase_Observada==1),:].sort_values(by="Prob_Anomalo",ascending=False)
seleccion

In [ ]:
img_path=seleccion.rutas.values[1]
prob=seleccion.Prob_Anomalo.values[1]

prob=np.round(prob*100,2)
img_size=(224,224)
img_array = resnet50.preprocess_input(get_img_array(img_path, size=img_size))
last_conv_layer_name="conv5_block3_out"

plt.figure(figsize=(9, 10))
plt.suptitle("Clase Observada: Anómalo\nClase Predicha: Anómalo ({p}%)".format(p=prob),y=0.75,size=15)

heatmap = make_gradcam_heatmap(img_array, modelo, last_conv_layer_name, pred_index=0)
plt.subplot(1,2,1)
plt.title("Mapa Características clase Normal")
plt.imshow(display_gradcam(img_path, heatmap,alpha=0.2))
plt.axis('off')
heatmap = make_gradcam_heatmap(img_array, modelo, last_conv_layer_name, pred_index=1)
plt.subplot(1,2,2)
plt.title("Mapa Características clase Anómala")
plt.imshow(display_gradcam(img_path, heatmap,alpha=0.2))
plt.axis('off')
plt.savefig('heatmap_1.png')
plt.show()

In [ ]:
img_path=seleccion.rutas.values[24]
prob=seleccion.Prob_Anomalo.values[24]
prob=np.round(prob*100,2)
img_size=(224,224)
img_array = resnet50.preprocess_input(get_img_array(img_path, size=img_size))
last_conv_layer_name="conv5_block3_out"

plt.figure(figsize=(9, 10))
plt.suptitle("Clase Observada: Anómalo\nClase Predicha: Anómalo ({p}%)".format(p=prob),y=0.75,size=15)

heatmap = make_gradcam_heatmap(img_array, modelo, last_conv_layer_name, pred_index=0)
plt.subplot(1,2,1)
plt.title("Mapa Características clase Normal")
plt.imshow(display_gradcam(img_path, heatmap,alpha=0.2))
plt.axis('off')
heatmap = make_gradcam_heatmap(img_array, modelo, last_conv_layer_name, pred_index=1)
plt.subplot(1,2,2)
plt.title("Mapa Características clase Anómala")
plt.imshow(display_gradcam(img_path, heatmap,alpha=0.2))
plt.axis('off')
plt.savefig('heatmap_1.png')
plt.show()

In [ ]:
img_path=seleccion.rutas.values[44]
prob=seleccion.Prob_Anomalo.values[44]
prob=np.round(prob*100,2)
img_size=(224,224)
img_array = resnet50.preprocess_input(get_img_array(img_path, size=img_size))
last_conv_layer_name="conv5_block3_out"

plt.figure(figsize=(9, 10))
plt.suptitle("Clase Observada: Anómalo\nClase Predicha: Anómalo ({p}%)".format(p=prob),y=0.75,size=15)

heatmap = make_gradcam_heatmap(img_array, modelo, last_conv_layer_name, pred_index=0)
plt.subplot(1,2,1)
plt.title("Mapa Características clase Normal")
plt.imshow(display_gradcam(img_path, heatmap,alpha=0.2))
plt.axis('off')
heatmap = make_gradcam_heatmap(img_array, modelo, last_conv_layer_name, pred_index=1)
plt.subplot(1,2,2)
plt.title("Mapa Características clase Anómala")
plt.imshow(display_gradcam(img_path, heatmap,alpha=0.2))
plt.axis('off')
plt.savefig('heatmap_1.png')
plt.show()

In [ ]:
seleccion=predicciones.loc[(predicciones.Clase_Observada==predicciones.Clase_Predicha) & (predicciones.Clase_Observada==0),:].sort_values(by="Prob_Normal",ascending=False)
seleccion

In [ ]:
img_path=seleccion.rutas.values[2]
prob=seleccion.Prob_Normal.values[2]
prob=np.round(prob*100,2)
img_size=(224,224)
img_array = resnet50.preprocess_input(get_img_array(img_path, size=img_size))
last_conv_layer_name="conv5_block3_out"

plt.figure(figsize=(9, 10))
plt.suptitle("Clase Observada: Normal\nClase Predicha: Normal ({p}%)".format(p=prob),y=0.75,size=15)

heatmap = make_gradcam_heatmap(img_array, modelo, last_conv_layer_name, pred_index=0)
plt.subplot(1,2,1)
plt.title("Mapa Características clase Normal")
plt.imshow(display_gradcam(img_path, heatmap,alpha=0.2))
plt.axis('off')
heatmap = make_gradcam_heatmap(img_array, modelo, last_conv_layer_name, pred_index=1)
plt.subplot(1,2,2)
plt.title("Mapa Características clase Anómala")
plt.imshow(display_gradcam(img_path, heatmap,alpha=0.2))
plt.axis('off')
plt.savefig('heatmap_1.png')
plt.show()

In [ ]:
img_path=seleccion.rutas.values[13]
prob=seleccion.Prob_Normal.values[13]
prob=np.round(prob*100,2)
img_size=(224,224)
img_array = resnet50.preprocess_input(get_img_array(img_path, size=img_size))
last_conv_layer_name="conv5_block3_out"

plt.figure(figsize=(9, 10))
plt.suptitle("Clase Observada: Normal\nClase Predicha: Normal ({p}%)".format(p=prob),y=0.75,size=15)

heatmap = make_gradcam_heatmap(img_array, modelo, last_conv_layer_name, pred_index=0)
plt.subplot(1,2,1)
plt.title("Mapa Características clase Normal")
plt.imshow(display_gradcam(img_path, heatmap,alpha=0.2))
plt.axis('off')
heatmap = make_gradcam_heatmap(img_array, modelo, last_conv_layer_name, pred_index=1)
plt.subplot(1,2,2)
plt.title("Mapa Características clase Anómala")
plt.imshow(display_gradcam(img_path, heatmap,alpha=0.2))
plt.axis('off')
plt.savefig('heatmap_1.png')
plt.show()

In [ ]:
seleccion=predicciones.loc[(predicciones.Clase_Observada!=predicciones.Clase_Predicha) & (predicciones.Clase_Observada==1),:].sort_values(by="Prob_Normal",ascending=False)
seleccion

In [ ]:
img_path=seleccion.rutas.values[0]
prob=seleccion.Prob_Normal.values[0]
prob=np.round(prob*100,2)
img_size=(224,224)
img_array = resnet50.preprocess_input(get_img_array(img_path, size=img_size))
last_conv_layer_name="conv5_block3_out"

plt.figure(figsize=(9, 10))
plt.suptitle("Clase Observada: Anómalo\nClase Predicha: Normal ({p}%)".format(p=prob),y=0.75,size=15)

heatmap = make_gradcam_heatmap(img_array, modelo, last_conv_layer_name, pred_index=0)
plt.subplot(1,2,1)
plt.title("Mapa Características clase Normal")
plt.imshow(display_gradcam(img_path, heatmap,alpha=0.2))
plt.axis('off')
heatmap = make_gradcam_heatmap(img_array, modelo, last_conv_layer_name, pred_index=1)
plt.subplot(1,2,2)
plt.title("Mapa Características clase Anómala")
plt.imshow(display_gradcam(img_path, heatmap,alpha=0.2))
plt.axis('off')
plt.savefig('heatmap_1.png')
plt.show()

In [ ]:
seleccion=predicciones.loc[(predicciones.Clase_Observada!=predicciones.Clase_Predicha) & (predicciones.Clase_Observada==0),:].sort_values(by="Prob_Anomalo",ascending=False)
seleccion

In [ ]:
mg_path=seleccion.rutas.values[0]
prob=seleccion.Prob_Anomalo.values[0]
prob=np.round(prob*100,2)
img_size=(224,224)
img_array = resnet50.preprocess_input(get_img_array(img_path, size=img_size))
last_conv_layer_name="conv5_block3_out"

plt.figure(figsize=(9, 10))
plt.suptitle("Clase Observada: Normal\nClase Predicha: Anómalo ({p}%)".format(p=prob),y=0.75,size=15)

heatmap = make_gradcam_heatmap(img_array, modelo, last_conv_layer_name, pred_index=0)
plt.subplot(1,2,1)
plt.title("Mapa Características clase Normal")
plt.imshow(display_gradcam(img_path, heatmap,alpha=0.2))
plt.axis('off')
heatmap = make_gradcam_heatmap(img_array, modelo, last_conv_layer_name, pred_index=1)
plt.subplot(1,2,2)
plt.title("Mapa Características clase Anómala")
plt.imshow(display_gradcam(img_path, heatmap,alpha=0.2))
plt.axis('off')
plt.savefig('heatmap_1.png')
plt.show()